# poetry in isolation

**import necessary libraries**

In [107]:
import random
import math
from numpy.random import uniform, normal, choice

In [108]:
# flat library deets
from flat import document, rgb, rgba, font, strike, shape
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
import numpy as np
from copy import copy

In [109]:
# helps display flat drawings inline
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

**two fxns to help get path commands & advance for a character**

In [110]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

**another fxn to help modify x & y coordinates of path**

In [111]:
def nudge(cmd, x, y):
    c = copy(cmd)
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += x
        c.y += y
    if type(cmd) in (quadto, curveto):
        c.x1 += x
        c.y1 += y
    if type(cmd) == curveto:
        c.x2 += x
        c.y2 += y
    return c

**define our special font**

In [112]:
f = font.open("./ArchivoBlack-Regular.ttf")

**load in a lonely poem**

In [113]:
lonely_poem = open("lonely-poem.txt").read()
print(lonely_poem)

the day is long
as heartbreak is inevitable
never sure when it will start
or when it will end
until the night comes
and i find another imaginary friend
to spend my lonely days with
until it has to end


**tend to the pieces of a lonely poem**

In [118]:
# create segments of text to play with
lonely_sentences = lonely_poem.split('\n')
first_lonely_sentence = lonely_sentences[0]
random_lonely_sentence = lonely_sentences[random.randrange(0,7)]

print("list of sentences: ")
print(lonely_sentences)
print("first sentence: " + first_lonely_sentence)
print("random sentence: " + random_lonely_sentence)

list of sentences: 
['the day is long', 'as heartbreak is inevitable', 'never sure when it will start', 'or when it will end', 'until the night comes', 'and i find another imaginary friend', 'to spend my lonely days with', 'until it has to end']
first sentence: the day is long
random sentence: never sure when it will start


## draw a sentence of a lonely poem one character at a time

In [119]:
s = first_lonely_sentence

In [132]:
page = document(700, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 0))
factor = 36 / f.density
cx = 0
for ch in s:
    glyph_path = glyphcommands(f, ch).scale(factor).translate(cx, 60)
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

## draw a sentence of a lonely poem as one mega path + widen glyphs

In [16]:
def widen_char(f, ch, width=-0.33):
    path_orig = glyphcommands(f, ch)
    path_cmds = []
    mid_x = advancefor(f, ch) * width
    for cmd in path_orig:
        if hasattr(cmd, 'x') and cmd.x > mid_x:
            path_cmds.append(nudge(cmd, x=200, y=0))
        else:
            path_cmds.append(cmd)
    path_mod = Path(path_cmds)
    return path_mod

In [130]:
s = lonely_sentences[random.randrange(0,7)]

page = document(2000, 90, 'pt').addpage()
pen = shape().stroke(rgba(0,0,0,255)).fill(rgba(0,0,0,255))
factor = 80/f.density
cx = 0
kerning = 15

for ch in s:
    glyph_path = widen_char(f, ch, uniform(0.18,1)).scale(factor).translate(cx, 90)
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor + kerning
show(page)

# attempt an alternative way to draw characters glyph by glyph

**split the first sentence up to play**

In [98]:
first_sentence = lonely_sentences[0]
print(first_sentence)

# # method 1
# def split(word):
#     return list(word)

# word = first_sentence
# word_list = split(word)
# print(split(word))

# method 2
def split(word):
    return [char for char in word]

word = first_sentence
word_list = split(word)
print(word_list)

the day is long
['t', 'h', 'e', ' ', 'd', 'a', 'y', ' ', 'i', 's', ' ', 'l', 'o', 'n', 'g']


**convert the list of characters to a list of paths**

In [104]:
list = word_list
for i in list:
    print(i)

Path([moveto(x=300.0000, y=-528.0000), lineto(x=412.0000, y=-528.0000), lineto(x=412.0000, y=-393.0000), lineto(x=300.0000, y=-393.0000), lineto(x=300.0000, y=-192.0000), quadto(x1=300.0000, y1=-156.0000, x=312.0000, y=-139.5000), quadto(x1=324.0000, y1=-123.0000, x=356.0000, y=-123.0000), lineto(x=412.0000, y=-123.0000), lineto(x=412.0000, y=-6.0000), quadto(x1=388.0000, y1=2.0000, x=350.0000, y=7.0000), quadto(x1=312.0000, y1=12.0000, x=284.0000, y=12.0000), quadto(x1=196.0000, y1=12.0000, x=148.5000, y=-20.0000), quadto(x1=101.0000, y1=-52.0000, x=101.0000, y=-129.0000), lineto(x=101.0000, y=-393.0000), lineto(x=27.0000, y=-393.0000), lineto(x=27.0000, y=-528.0000), lineto(x=109.0000, y=-528.0000), lineto(x=152.0000, y=-688.0000), lineto(x=300.0000, y=-688.0000), lineto(x=300.0000, y=-528.0000), closepath])
Path([moveto(x=259.0000, y=-473.0000), quadto(x1=291.0000, y1=-505.0000, x=334.0000, y=-522.5000), quadto(x1=377.0000, y1=-540.0000, x=424.0000, y=-540.0000), quadto(x1=516.0000,

**loop through our new list of charcters in the umbrella word list** (unsucessfully)

In [121]:
path_orig = word_list

index = 0
while index < len(word_list):
    path_orig[index] = glyphcommands(f, word_list[index])
    index = index + 1
print(word_list)

TypeError: ord() expected string of length 1, but Path found

In [55]:
path_orig = word_list
path_cmds = []

index = 0

# iterate through list to define glyphs for paths
while index < len(word_list):
    path_orig[index] = glyphcommands(f, word_list[index])
    index = index + 1
    
# draw paths
for cmd in path_orig:
    if hasattr(cmd, 'x'):
        path_cmds.append(nudge(cmd, x=cmd.x*0.8, y=0))
    else:
        path_cmds.append(cmd)
path_mod = Path(path_cmds)

# display & style on page
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))

# place glyphs on page
while index < len(word_list):
    # page.place(pen.path(path_orig[index].scale(144/f.density).translate(0, 144)))
    page.place(pen.path(path_mod.scale(144/f.density).translate(144, 144)))


show(page)
        
    

TypeError: ord() expected string of length 1, but Path found

# references

In [124]:
# the following two cells are for reference of how this works on an individual character

# part 1 stretch a character
g_path_orig = glyphcommands(f, first_lonely_sentence[-1])
g_path_cmds = []
for cmd in g_path_orig:
    if hasattr(cmd, 'x'):
        g_path_cmds.append(nudge(cmd, x=cmd.x*0.8, y=0))
    else:
        g_path_cmds.append(cmd)
g_path_mod = Path(g_path_cmds)
        

In [125]:
# part 2 display a stretched character
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
# page.place(pen.path(g_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(g_path_mod.scale(144/f.density).translate(144, 144)))
show(page)